In [7]:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
import pandas as pd
import numpy as np
from time import time
from sklearn.datasets import load_digits

In [3]:
data = pd.read_csv("datasets/income.csv")
data

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income >50K
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,0
32557,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,1
32558,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,0
32559,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,0


In [4]:
data["education"] = data["education"].replace(
    {
        'Preschool': 'dropout',
        '10th': 'dropout',
        '11th': 'dropout',
        '12th': 'dropout',
        '1st-4th': 'dropout',
        '5th-6th': 'dropout',
        '7th-8th': 'dropout',
        '9th': 'dropout',
        'HS-Grad': 'HighGrad',
        'HS-grad': 'HighGrad',
        'Some-college': 'CommunityCollege',
        'Assoc-acdm': 'CommunityCollege',
        'Assoc-voc': 'CommunityCollege',
        'Bachelors': 'Bachelors',
        'Masters': 'Masters',
        'Prof-school': 'Masters',
        'Doctorate': 'Doctorate'
    }
)
data["marital-status"] = data["marital-status"].replace(
    {
        'Never-married': 'NotMarried',
        'Married-AF-spouse': 'Married',
        'Married-civ-spouse': 'Married',
        'NotMarried': 'NotMarried',
        'Separated': 'Separated',
        'Divorced': 'Separated',
        'Widowed': 'Widowed',
    }
)
ordinal_columns={
    "education": ["dropout", "HighGrad", "CommunityCollege", "Bachelors", "Masters", "Doctorate"],
}
data["native-country"] = data["native-country"].replace(
{"Canada": "North America", "Cuba": "South America", "Dominican-Republic": "South America", "El-Salvador": "South America", "Guatemala": "South America",
                   "Haiti": "South America", "Honduras": "South America", "Jamaica": "South America", "Mexico": "South America", "Nicaragua": "South America",
                   "Outlying-US(Guam-USVI-etc)": "North America", "Puerto-Rico": "North America", "Trinadad&Tobago": "South America",
                   "United-States": "North America", "Cambodia": "Asia", "China": "Asia", "Hong": "Asia", "India": "Asia", "Iran": "Asia", "Japan": "Asia", "Laos": "Asia",
          "Philippines": "Asia", "Taiwan": "Asia", "Thailand": "Asia", "Vietnam": "Asia", "Columbia": "South America", "Ecuador": "South America", "Peru": "South America", "England": "West Europe", "France": "West Europe", "Germany": "West Europe", "Greece": "West Europe", "Holand-Netherlands": "West Europe",
            "Ireland": "West Europe", "Italy": "West Europe", "Portugal": "West Europe", "Scotland": "West Europe", "Hungary": "East Europe", "Poland": "East Europe", "Yugoslavia": "East Europe", "South": "?"})
data["capital"] = data["capital-gain"]-data["capital-loss"]
data.drop(["capital-gain", "capital-loss"], axis=1, inplace=True)
data["education-num"] = data["education-num"].astype(float)

In [5]:
from sklearn.model_selection import RepeatedStratifiedKFold
from automl.search.automl import AutoML

am = AutoML(
    random_state=42,
    level="common",
    #target_metric="f1",
    trainer_config={
        #"secondary_level": ComponentLevel.UNCOMMON,
        "cache": True,
        "early_stopping": True,
        "return_test_scores_during_tuning": True,
        "tuning_time": 60*1,
        "stacking_level": 1,
        "tune_kwargs": {"max_concurrent": 2, "trainable_n_jobs": 4}
    },
    #cv=RepeatedStratifiedKFold(random_state=420, n_repeats=2, n_splits=4)
    cv=4,
)

In [6]:
t_s = time()
am.fit(data.drop("income >50K", axis=1), data["income >50K"], ordinal_columns={"education":ordinal_columns})
#am.fit(data.drop("Purchase", axis=1), data["Purchase"])
#am.fit(data.drop("default", axis=1), data["default"], ordinal_columns={"EDUCATION": list(sorted(data["EDUCATION"].unique()))})
#am.fit(data.drop("quality", axis=1), data["quality"]>=6)
#am.fit(data.drop("target", axis=1), data["target"])
#am.fit(data.drop("target", axis=1), data["target"], ordinal_columns=ordinal_columns)
#am.fit(X_train, y_train, X_test=X_test, y_test=y_test)
t_end = time() - t_s

2021-08-18 20:28:54,107	INFO services.py:1245 -- View the Ray dashboard at http://127.0.0.1:8265
``multivariate`` option is an experimental feature. The interface can change in the future.
``group`` option is an experimental feature. The interface can change in the future.
``constant_liar`` option is an experimental feature. The interface can change in the future.
[I 2021-08-18 20:28:55,734] A new study created in memory with name: optuna


FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best validation score',
         …

2021-08-18 20:28:56,087	WARNING worker.py:1189 -- It looks like you're creating a detached actor in an anonymous namespace. In order to access this actor in the future, you will need to explicitly connect to this namespace with ray.init(namespace="a0a8831c-355b-4445-9d40-223ac55929c3", ...)


suggest e9396efa, 12
e9396efa final suggestion by 0: {'ColumnScaling__ScalerNormalizer': 'CombinedScalerTransformer(sparse_threshold=0.3, n_jobs=1, transformer_weights=None, verbose=False)', 'Estimator': 'LogisticRegression(penalty=elasticnet, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=balanced, random_state=0, solver=saga, max_iter=200, multi_class=auto, verbose=0, warm_start=False, n_jobs=None, l1_ratio=0)', 'FeatureSelector': 'Passthrough(None)', 'Estimator__LogisticRegression__C': 1.0, 'Estimator__LogisticRegression__l1_ratio': 1, 'Estimator__LogisticRegression__class_weight': 'balanced', 'Imputer__CombinedSimpleImputer__categorical_strategy': 'most_frequent', 'Imputer': 'CombinedSimpleImputer(numeric_strategy=median, numeric_fill_value=0, categorical_strategy=most_frequent, categorical_fill_value=missing_value, verbose=0, copy=True, n_jobs=1, transformer_weights=None)', 'Imbalance': 'Passthrough(None)', 'ColumnOrdinal__OrdinalEncoder': 'pa

suggest e9396efb, 11
e9396efb final suggestion by 0: {'ColumnScaling__ScalerNormalizer': 'CombinedScalerTransformer(sparse_threshold=0.3, n_jobs=1, transformer_weights=None, verbose=False)', 'Estimator': 'LogisticRegression(penalty=elasticnet, dual=False, tol=0.0001, C=1.0, fit_intercept=True, intercept_scaling=1, class_weight=balanced, random_state=0, solver=saga, max_iter=200, multi_class=auto, verbose=0, warm_start=False, n_jobs=None, l1_ratio=0)', 'FeatureSelector': 'Passthrough(None)', 'Estimator__LogisticRegression__C': 1.0, 'Estimator__LogisticRegression__l1_ratio': 0.15, 'Estimator__LogisticRegression__class_weight': 'balanced', 'Imputer__CombinedSimpleImputer__categorical_strategy': 'most_frequent', 'Imputer': 'CombinedSimpleImputer(numeric_strategy=median, numeric_fill_value=0, categorical_strategy=most_frequent, categorical_fill_value=missing_value, verbose=0, copy=True, n_jobs=1, transformer_weights=None)', 'Imbalance': 'Passthrough(None)', 'ColumnOrdinal__OrdinalEncoder': 

...

(pid=4089375) elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


(pid=4089374) scoring took 22.606647729873657
(pid=4089374) {'accuracy': 0.8335585308930107, 'balanced_accuracy': 0.8487539579818337, 'roc_auc': 0.929609253505641, 'precision': 0.6066267183644695, 'recall': 0.878061224489796, 'specificity': 0.8194466914738715, 'f1': 0.7175317907025224, 'matthews_corrcoef': 0.625857012384112}


(pid=4089374) elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
(pid=4089375) elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
(pid=4089375) elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


(pid=4089375) scoring took 43.69008708000183
(pid=4089375) {'accuracy': 0.833681365925562, 'balanced_accuracy': 0.8497058959484134, 'roc_auc': 0.9313743318068208, 'precision': 0.60646521433591, 'recall': 0.8806122448979592, 'specificity': 0.8187995469988675, 'f1': 0.7182688306283812, 'matthews_corrcoef': 0.6271045148576804}


,Pipeline,Test accuracy,Test balanced_accuracy,Test roc_auc,Test precision,Test recall,Test specificity,Test f1,Test matthews_corrcoef,Test optimized_precision,...,Validation precision,Validation recall,Validation specificity,Validation f1,Validation matthews_corrcoef,Validation optimized_precision,Stacking Level,Total Time (s),Estimator Fit Time (s),Dataset Fraction
Id,,,,,,,,,,,,,,,,,,,,,
0_StackFSSelectFromModel,PandasStackingClassifier,0.833927,0.848648,0.930964,0.607420,0.877041,0.820256,0.717745,0.626028,0.800471,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1.0
0_Stack,PandasStackingClassifier,0.834173,0.849507,0.930545,0.607546,0.879082,0.819932,0.718515,0.627260,0.799359,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,1.0
1_StackFSSelectFromModel,PandasStackingClassifier,0.833559,0.848754,0.929609,0.606627,0.878061,0.819447,0.717532,0.625857,0.799029,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1.0
ed312d4a,"CatBoostClassifierWithAutoCatFeatures w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.833067,0.848256,0.930964,0.605847,0.877551,0.818961,0.716816,0.624861,0.798532,...,0.604571,0.859209,0.821727,0.709741,0.613655,0.808456,0,20.371470,16.857204,1.0
1_Stack,PandasStackingClassifier,0.833681,0.849706,0.931374,0.606465,0.880612,0.818800,0.718269,0.627105,0.797308,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1.0
2dfa2e44,"CatBoostClassifierWithAutoCatFeatures w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder",0.829628,0.845469,0.930831,0.600140,0.876020,0.814917,0.712300,0.618757,0.793492,...,0.600055,0.845606,0.821188,0.701966,0.602147,0.812418,1,24.216765,19.409542,1.0
1_VotingSoft,PandasVotingClassifier,0.833436,0.851112,0.931250,0.605373,0.885204,0.817020,0.719022,0.628614,0.793380,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1.0
1_VotingSoftByMetric,PandasVotingClassifier,0.833436,0.851112,0.931252,0.605373,0.885204,0.817020,0.719022,0.628614,0.793380,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1.0
2750cad0,"LogisticRegression w/ PandasSimpleCategoricalImputer, PandasSimpleImputer, PandasOrdinalEncoder, KFoldCatBoostEncoder, PandasStandardScaler, PandasQuantileTransformer",0.831102,0.848181,0.929609,0.601952,0.881122,0.815240,0.715262,0.623218,0.792265,...,0.602420,0.857679,0.820433,0.707727,0.610853,0.807207,1,12.044163,9.192110,1.0


In [8]:
a = am.get_pipeline_by_id("1_StackFSSelectFromModel")

In [9]:
a

Pipeline(steps=[('PrepareData',
                 PrepareDataFrame(ordinal_columns={'education': {'education': ['dropout',
                                                                               'HighGrad',
                                                                               'CommunityCollege',
                                                                               'Bachelors',
                                                                               'Masters',
                                                                               'Doctorate']}})),
                ('Ensemble',
                 PandasStackingClassifier(cv=StratifiedKFold(n_splits=4,
        random_state=RandomState(MT19937) at 0x7F838E61A440, shuffle=False),
                                          estimators=[('meta-1_2750cad0',
                                                       PandasStackingCla...
                                                                                

In [ ]:
x = a.fit_transform(am.X_, am.y_)

In [ ]:
x

In [ ]:
x.dtypes

In [ ]:
am.X_test_

In [ ]:
a.named_steps["ColumnEncodingScaling"].transformers[0][1].transformers_

In [ ]:
a.named_steps["ColumnEncodingScaling"].transformers[0][1].transformers_

In [ ]:
a.score(am.X_test_, am.y_test_)

In [ ]:
am.visualize_pipeline("65a71b94")

In [ ]:
am.results_.sort_values("Test accuracy", ascending=False)[:60]

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

lr = LogisticRegression(random_state=42).fit(X_train, y_train)
dt = DecisionTreeClassifier(random_state=42).fit(X_train, y_train)

In [ ]:
predictions = [lr.predict_proba(X_train), dt.predict_proba(X_train)]

In [ ]:
from automl_models.components.estimators.ensemble import PandasStackingClassifier
stack = PandasStackingClassifier(estimators=[("lr", lr), ("dt", dt)], final_estimator=LogisticRegression(random_state=42)).fit(X_train, y_train)

In [ ]:
p = stack.predict_proba(X_test)

In [ ]:
stack._saved_test_predictions=predictions
pp = stack.predict_proba(X_test)

In [ ]:
p == pp

In [ ]:
am.results_.loc["0_VotingUniform"]

In [ ]:
p = am.get_pipeline_by_id("0_VotingUniform")

In [ ]:
p.score(am.X_test_, am.y_test_)

In [ ]:
p._final_estimator

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer, load_iris, make_classification
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, clone, ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from deslib.des import KNORAE, METADES, KNORAU
import numpy as np

In [ ]:
rfc = RandomForestClassifier(random_state=42, oob_score=True, bootstrap=True)
X, y = load_breast_cancer(return_X_y=True)


In [ ]:
data = pd.read_csv("datasets/wine.csv")

In [ ]:
X = data.drop("type", axis=1).to_numpy()
y = LabelEncoder().fit_transform(data["type"].to_numpy())

In [ ]:
cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

In [ ]:
class DESCV(ClassifierMixin, BaseEstimator):
    def __init__(self, des, classifier_pool, cv):
        self.des = des
        self.classifier_pool = classifier_pool
        self.cv = cv
    def fit(self, X, y):
        self.le_ = LabelEncoder().fit(y)
        self.classes_ = self.le_.classes_
        transformed_y = self.le_.transform(y)

        des = []
        for train_ind, test_ind in self.cv.split(X, y):
            rfc_ = clone(self.classifier_pool).fit(X[train_ind], y[train_ind])
            des_ = clone(self.des)
            des_.set_params(pool_classifiers=rfc_)
            des_.fit(X[test_ind], y[test_ind])
            des.append(des_)
        self.estimators_ = des

    def _predict(self, X):
        """Collect results from clf.predict calls."""
        return np.asarray([est.predict(X) for est in self.estimators_]).T

    def _collect_probas(self, X):
        """Collect results from clf.predict calls."""
        return np.asarray([clf.predict_proba(X) for clf in self.estimators_])

    def predict(self, X):
        maj = np.argmax(self.predict_proba(X), axis=1)
        maj = self.le_.inverse_transform(maj)

        return maj

    def predict_proba(self, X):
        """Predict class probabilities for X in 'soft' voting."""
        #check_is_fitted(self)
        avg = np.average(self._collect_probas(X), axis=0)
        return avg

In [ ]:
X, y = make_classification(n_samples=7000, random_state=42)

In [ ]:
descv = DESCV(KNORAE(None, DFP=True, random_state=42), RandomForestClassifier(random_state=42, oob_score=True, bootstrap=True), cv)

In [ ]:
desucv = DESCV(KNORAU(None, DFP=True, random_state=42), RandomForestClassifier(random_state=42, oob_score=True, bootstrap=True), cv)

In [ ]:
metadescv = DESCV(METADES(None, DFP=True, random_state=42), RandomForestClassifier(random_state=42, oob_score=True, bootstrap=True), cv)

In [ ]:
%%time
np.mean(cross_val_score(rfc, X, y, scoring="roc_auc_ovr_weighted"))

In [ ]:
np.mean(cross_val_score(rfc, X, y, scoring="f1", error_score="raise"))

In [ ]:
%%time
np.mean(cross_val_score(descv, X, y, scoring="roc_auc_ovr_weighted", error_score="raise"))

In [ ]:
np.mean(cross_val_score(descv, X, y, scoring="f1"))

In [ ]:
%%time
np.mean(cross_val_score(desucv, X, y, scoring="roc_auc_ovr_weighted", error_score="raise"))

In [ ]:
np.mean(cross_val_score(desucv, X, y, scoring="f1", error_score="raise"))

In [ ]:
%%time
np.mean(cross_val_score(metadescv, X, y, scoring="roc_auc_ovr_weighted", error_score="raise"))

In [ ]:
np.mean(cross_val_score(metadescv, X, y, scoring="f1", error_score="raise"))

In [ ]:
rfc.fit(X, y)
rfc.oob_decision_function_

In [ ]:
np.argmax(rfc.oob_decision_function_, axis=1)

In [ ]:
rfc.oob_decision_function_.shape